# Preps

## Importing Independencieisausius

In [ ]:
import pandas as pd
import numpy as np

# !pip install -q darts
# !pip install -q sklearn

import darts
from darts.metrics import mape, mae, rmse
from darts import TimeSeries, concatenate
from darts.models import TFTModel, RNNModel, TCNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.statistics import check_seasonality, plot_acf, extract_trend_and_seasonality
import torch

%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import shutil
import tables
from statsmodels.tsa.stattools import adfuller

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

!pip install git+https://github.com/PyTorchLightning/pytorch-lightning -q
!pip install optuna -q
import pytorch_lightning as pl
print(pl.__version__)
# !pip install pytorch-lightning -q

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest
from torchmetrics import MeanAbsolutePercentageError, MeanSquaredError
import time

In [ ]:
# !git clone https://github.com/AfterRain007/Skripsi
# !git clone https://github.com/katemurraay/tsa_crt

## Importing Function

In [ ]:
#Evaluate Model
def eval_model(n, series, pred_series, x):
    ## plot time series, limited to forecast horizon
    plt.figure(figsize=(9, 6))

    series.plot(label="actual")                                       # plot actual
    pred_series.plot(central_quantile="mean", label="expected")              # plot "mean" or median=0.5

    plt.title("TFT: test set (MAPE: {:.2f}%)".format(mape_))
    plt.legend()
    plt.savefig(f'/content/drive/MyDrive/Plot Images/Experiment/imageTFT-{x}.png')
    plt.close()

In [ ]:
#Do Differencing Transformation to All Data
def differT(df):
    columns_to_diff = ['price', 'volume']
    
    for column in columns_to_diff:
        print(f"Transforming {column}")
        df[column] = df[column].diff()
    
    df = df.iloc[1:].copy()
    
    return df

#Inversing Back Differenced Data

def inverse_differenced_dataset(df, diff_vals, l = 0, df_start = 0):
  """
  Inverses the Difference on a Dataset
  :param  pd.DataFrame df: the orginial DataFrame from csv file
          list diff_vals:  List of Differenced Values
  :return np.array inverted_values: Array of values with Difference removed
  """
  invert = list()
  target = 'price'
  if df_start == 0:
      if l == 0: df_start = len(df) - len(diff_vals) -1
      else: df_start = len(df) - l -1

  for i in range(len(diff_vals)):
      value =  diff_vals[i] + df[target][df_start + i]
      invert.append(value)
  inverted_values = np.array(invert)
  return inverted_values

def testingDiff(model):
  global predict
  pred = model.predict(len(ts_ttest))
  pred = scalerP.inverse_transform(pred)
  done = inverse_differenced_dataset(dfPure.pd_dataframe(), pred.pd_dataframe()['price'].tolist())
  predict = pd.DataFrame(done)[1:]

  predict.index = test[:len(test)-1].time_index
  predict = TimeSeries.from_dataframe(predict)
  smape_ = smape(predict, test, n_jobs=-1, verbose=True)
  mape_ = mape(predict, test, n_jobs=-1, verbose=True)
  return smape_, mape_

In [ ]:
def anomaly_search(df):
  anomaly_inputs = ['price', 'volume']
  model_IF = IsolationForest(contamination=float(0.1),random_state=42069)
  model_IF.fit(df[anomaly_inputs])
  df['anomaly_scores'] = model_IF.decision_function(df[anomaly_inputs])
  df['anomaly'] = model_IF.predict(df[anomaly_inputs])
  return df

def replaceOutlierMA(df, i):
    lenWin = i
    df = anomaly_search(df)
    df['anomaly'] = df['anomaly'].map({1:0, -1:1})
    dfAnomaly  = df[df['anomaly'] == 1]
    
    df['repVolume'] = df['volume'].rolling(window=lenWin).mean()
    df['repPrice'] = df['price'].rolling(window=lenWin).mean()
    
    dfAnomaly = dfAnomaly[dfAnomaly.index < df.iloc[int(len(df)*.8)].name].sort_values(by = 'anomaly_scores', ascending = True)
    dfAnomaly = dfAnomaly.iloc[:int(len(dfAnomaly)*.1)]
    
    df2 = df.copy()
    df2.loc[dfAnomaly.index, 'price']  =  df2['repPrice'].loc[dfAnomaly.index].values
    df2.loc[dfAnomaly.index, 'volume'] = df2['repVolume'].loc[dfAnomaly.index].values
    df2.drop(['anomaly_scores', 'anomaly', 'repPrice', 'repVolume'], axis = 1, inplace = True)

    df2 = differT(df2)
    return df2

def replaceOutlierBP(df):
    # Calculate the IQR (Interquartile Range)
    columns = ['price', 'volume']
    for column in columns:
        
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
    
        # Define the lower and upper bounds for outlier detection
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
    
        # Identify outliers
        outliers_upper = (df[column] > upper_bound)
        outliers_under = (df[column] < lower_bound)
    
        # Handle outliers by replacing them with the bound value
        df.loc[outliers_upper, column] = upper_bound
        df.loc[outliers_under, column] = lower_bound

    df = differT(df)
    return df

# Optuna Griddy

## Function for Griddy

In [ ]:
def createTimeseries(df):
    global ts_t
    ts_P = TimeSeries.from_dataframe(df[['price']])
    ts_covF = TimeSeries.from_dataframe(df[['sen', 'volume', 'trend']])
    
    SPLIT = .8
    # train/test split and scaling of target variable
    ts_train, ts_test = ts_P.split_after(SPLIT)
    
    scalerP = Scaler()
    scalerP.fit_transform(ts_train)
    ts_ttrain = scalerP.transform(ts_train)
    ts_ttest = scalerP.transform(ts_test)
    ts_t = scalerP.transform(ts_P)
    
    # make sure data are of type float
    ts_t = ts_t.astype(np.float32)
    ts_ttrain = ts_ttrain.astype(np.float32)
    ts_ttest = ts_ttest.astype(np.float32)
    
    # train/test split and scaling of feature covariates
    covF_train, covF_test = ts_covF.split_after(SPLIT)
    
    scalerF = Scaler()
    scalerF.fit_transform(covF_train)
    covF_ttrain = scalerF.transform(covF_train)
    covF_ttest = scalerF.transform(covF_test)
    covF_t = scalerF.transform(ts_covF)
    
    # make sure data are of type float
    covF_ttrain = covF_ttrain.astype(np.float32)
    covF_ttest = covF_ttest.astype(np.float32)
    
    # feature engineering - create time covariates: hour, weekday, month, year, country-specific holidays
    covT = datetime_attribute_timeseries(ts_P.time_index, attribute="day", one_hot=False, add_length=len(covF_ttrain))
    covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="week"))
    covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="month"))
    covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="year"))
    covT = covT.stack(TimeSeries.from_times_and_values(times=covT.time_index, values=np.arange(len(ts_P) + len(covF_ttrain)), columns=["linear_increase"]))
    covT = covT.add_holidays(country_code="US")
    covT = covT.astype(np.float32)
    
    # train/test split
    covT_train, covT_test = covT.split_after(ts_train.end_time())
    
    # rescale the covariates: fitting on the training set
    scalerT = Scaler()
    scalerT.fit(covT_train)
    covT_ttrain = scalerT.transform(covT_train)
    covT_ttest = scalerT.transform(covT_test)
    covT_t = scalerT.transform(covT)
    
    covT_ttrain = covT_ttrain.astype(np.float32)
    covT_ttest  = covT_ttest.astype(np.float32)
    covT_t      = covT_t.astype(np.float32)
    
    # combine feature and time covariates along component dimension: axis=1
    ts_cov = ts_covF.concatenate( covT.slice_intersect(ts_covF), axis=1 )                      # unscaled F+T
    cov_t = covF_t.concatenate( covT_t.slice_intersect(covF_t), axis=1 )                       # scaled F+T
    cov_ttrain = covF_ttrain.concatenate( covT_ttrain.slice_intersect(covF_ttrain), axis=1 )   # scaled F+T training set
    cov_ttest = covF_ttest.concatenate( covT_ttest.slice_intersect(covF_ttest), axis=1 )       # scaled F+T test set
    
    return ts_ttrain, ts_ttest, cov_t

In [ ]:
# define objective function
def objectiveLSTM(trial):
    global mape_, mae_
    # select input and output chunk lengths
    input_chunk_length = trial.suggest_int("input_chunk_length", 1, 31)
    training_length = trial.suggest_int("training_length", input_chunk_length + 1, 62)

    # Other hyperparameters
    hidden_dim = trial.suggest_int("hidden_dim", 8, 32)
    n_rnn_layers = trial.suggest_categorical("n_rnn_layers", [1, 2, 3])
    dropout = trial.suggest_float("dropout", 0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    lr = trial.suggest_float("lr", 1e-3, 5e-3)
    
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }

    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    # build the LSTM model
    model = RNNModel(
        model = "LSTM",
        hidden_dim = hidden_dim,
        n_rnn_layers = n_rnn_layers,
        dropout = dropout,
        batch_size = batch_size,
        training_length = training_length,
        input_chunk_length = input_chunk_length,
        optimizer_kwargs = {"lr": lr},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        torch_metrics = MeanAbsolutePercentageError(),
        log_tensorboard = True,
        random_state = 42069,
        force_reset = True,
        save_checkpoints = False,
        pl_trainer_kwargs = pl_trainer_kwargs
    )

    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    # model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])

    # train the model
    model.fit(ts_ttrain,                    # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)

    # Evaluate how good it is on the validation set
    pred = model.predict(len(ts_ttest))
    
    # Calculate Metrics
    mape_ = mape(ts_ttest, pred)
    mae_ = mae(ts_ttest, pred)
    rmse_ = rmse(ts_ttest, pred)

    return rmse_ if rmse_ != np.nan else float("inf")

In [ ]:
# define objective function
def objectiveGRU(trial):
    global mape_, mae_
    # select input and output chunk lengths
    input_chunk_length = trial.suggest_int("input_chunk_length", 1, 31)
    training_length = trial.suggest_int("training_length", input_chunk_length + 1, 62)

    # Other hyperparameters
    hidden_dim = trial.suggest_int("hidden_dim", 8, 32)
    n_rnn_layers = trial.suggest_categorical("n_rnn_layers", [1, 2, 3])
    dropout = trial.suggest_float("dropout", 0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    lr = trial.suggest_float("lr", 1e-3, 5e-3)
    
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }

    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    # build the GRU model
    model = RNNModel(
        model = "GRU",
        hidden_dim = hidden_dim,
        n_rnn_layers = n_rnn_layers,
        dropout = dropout,
        batch_size = batch_size,
        training_length = training_length,
        input_chunk_length = input_chunk_length,
        optimizer_kwargs = {"lr": lr},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        torch_metrics = MeanAbsolutePercentageError(),
        log_tensorboard = True,
        random_state = 42069,
        force_reset = True,
        save_checkpoints = False,
        pl_trainer_kwargs = pl_trainer_kwargs
    )

    # when validating during training, we can use a slightly longer validation
    # set which also contains the first input_chunk_length time steps
    # model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])

    # train the model
    model.fit(ts_ttrain,                    # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)

    # Evaluate how good it is on the validation set
    pred = model.predict(len(ts_ttest))
    
    # Calculate Metrics
    mape_ = mape(ts_ttest, pred)
    mae_ = mae(ts_ttest, pred)
    rmse_ = rmse(ts_ttest, pred)

    return rmse_ if rmse_ != np.nan else float("inf")

In [ ]:
# define objective function
def objectiveTFT(trial):
    global mape_, mae_
    # select input and output chunk lengths
    in_len = trial.suggest_int("in_len", 1, 31)
    out_len = trial.suggest_int("out_len", 1, 31)

    # Other hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 8, 32)
    lstm_layers = trial.suggest_categorical("lstm_layers", [1, 2, 3])
    num_attention_heads = trial.suggest_int("num_attention_heads", 2, 8)
    dropout = trial.suggest_float("dropout", 0, 0.3)
    hidden_continuous_size = trial.suggest_int("hidden_continuous_size", 6, 10)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    full_attention = trial.suggest_categorical("full_attention", [True, False])
    lr = trial.suggest_float("lr", 1e-3, 5e-3)
    
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }

    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    # build the TFT model
    model = TFTModel(
        input_chunk_length = in_len,
        output_chunk_length = out_len,
        hidden_size = hidden_size,
        lstm_layers = lstm_layers,
        num_attention_heads = num_attention_heads,
        dropout = dropout,
        hidden_continuous_size = hidden_continuous_size,
        use_static_covariates = True,
        batch_size = batch_size,
        optimizer_kwargs = {'lr': lr},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        likelihood = None, 
        loss_fn = torch.nn.MSELoss(),
        full_attention = full_attention,
        torch_metrics = MeanAbsolutePercentageError(),
        random_state = 42069,
        force_reset= True,
        pl_trainer_kwargs = pl_trainer_kwargs,
        add_relative_index= False,
        # add_encoders = None,
        # likelihood=QuantileRegression(quantiles=QUANTILES)
    )

    # train the model
    model.fit(ts_ttrain,             # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)
    
    # Evaluate how good it is on the validation set
    pred = model.predict(len(ts_ttest))
    
    # Calculate Metrics
    mape_ = mape(ts_ttest, pred)
    mae_ = mae(ts_ttest, pred)
    rmse_ = rmse(ts_ttest, pred)

    return rmse_ if rmse_ != np.nan else float("inf")

In [ ]:
# define objective function
def objectiveTCN(trial):
    global mape_, mae_
    # select input and output chunk lengths
    in_len = trial.suggest_int("in_len", 2, 31)
    out_len = trial.suggest_int("out_len", 1, in_len - 1)

    # Other hyperparameters
    kernel_size = trial.suggest_int("kernel_size", 1, in_len - 1)
    num_filters = trial.suggest_int("num_filters", 2, 8)
    weight_norm = trial.suggest_categorical("weight_norm", [True, False])
    dilation_base = trial.suggest_int("dilation_base", 1, 5)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    dropout = trial.suggest_float("dropout", 0, 0.3)
    lr = trial.suggest_float("lr", 1e-3, 5e-3)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }

    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    # print(f"{in_len} - {out_len} - {kernel_size} - {num_filters} - {weight_norm} - {dilation_base} - {num_layers} - {dropout} - {lr}")
    # build the TCN model
    model = TCNModel(
        input_chunk_length = in_len,
        output_chunk_length = out_len,
        kernel_size = kernel_size,
        num_filters = num_filters,
        weight_norm = weight_norm,
        dilation_base = dilation_base,
        num_layers = num_layers,
        dropout = dropout,
        batch_size = batch_size,
        optimizer_kwargs = {'lr': lr},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        likelihood = None, 
        loss_fn = torch.nn.MSELoss(),
        torch_metrics = MeanAbsolutePercentageError(),
        random_state = 42069,
        force_reset= True,
        pl_trainer_kwargs = pl_trainer_kwargs,
    )

    # train the model
    model.fit(
        series=ts_ttrain,
        val_series=ts_ttest,
        past_covariates=cov_t,
        val_past_covariates=cov_t,
    )
    
    # Evaluate how good it is on the validation set
    pred = model.predict(len(ts_ttest))
    
    # Calculate Metrics
    mape_ = mape(ts_ttest, pred)
    mae_ = mae(ts_ttest, pred)
    rmse_ = rmse(ts_ttest, pred)

    return rmse_ if rmse_ != np.nan else float("inf")

In [ ]:
# for convenience, print some optimization trials information
def print_callback(study, trial):
    global dfResult
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

    temp = trial.params
    temp.update({"RMSE" : trial.value, "MAE" : mae_, "MAPE" : mape_})
    results_list.append(temp)
    dfResult = pd.DataFrame(results_list)

## OMG GWIDDY

### TFT

In [ ]:
timeTFT = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_30.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_BP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_MAnP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_BPnP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['trend', 'day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_MAonlyTrend.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataExperiment2.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['sen', 'trend', 'day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_MAexperiment.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['sen', 'trend', 'day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTemp = time.time() - start

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_MAdelVol.csv")

In [ ]:
print(timeTemp)

In [ ]:
resultFinale.sort_values(by = "RMSE")

In [ ]:
pd.DataFrame(timeTFT).to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/timeTFT.xlsx")

### LSTM

In [ ]:
timeLSTM = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveLSTM, n_trials=100, callbacks=[print_callback])
timeLSTM.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultLSTM_30.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveLSTM, n_trials=100, callbacks=[print_callback])
timeLSTM.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultLSTM_BP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveLSTM, n_trials=100, callbacks=[print_callback])
timeLSTM.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultLSTM_MAnP.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveLSTM, n_trials=100, callbacks=[print_callback])
timeLSTM.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultLSTM_BPnP.csv")

In [ ]:
pd.DataFrame(timeLSTM).to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/timeLSTM.xlsx")

### GRU

In [ ]:
timeGRU = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveGRU, n_trials=100, callbacks=[print_callback])
timeGRU.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultGRU_30.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveGRU, n_trials=100, callbacks=[print_callback])
timeGRU.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultGRU_BP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveGRU, n_trials=100, callbacks=[print_callback])
timeGRU.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultGRU_MAnP.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveGRU, n_trials=100, callbacks=[print_callback])
timeGRU.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultGRU_BPnP.csv")

In [ ]:
pd.DataFrame(timeGRU).to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/timeGRU.xlsx")

### TCN

In [ ]:
from darts.models import TCNModel
timeTCN = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTCN, n_trials=100, callbacks=[print_callback])
timeTCN.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultTCN_30.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTCN, n_trials=100, callbacks=[print_callback])
timeTCN.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultTCN_BP.csv")

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTCN, n_trials=100, callbacks=[print_callback])
timeTCN.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultTCN_MAnP.csv")

In [ ]:
resultFinale = pd.DataFrame()
results_list = []

df2 = replaceOutlierBP(df)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTCN, n_trials=100, callbacks=[print_callback])
timeTCN.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/resultTCN_BPnP.csv")

In [ ]:
pd.DataFrame(timeTCN).to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/timeTCN.xlsx")

## Experiment Without Covariates

In [ ]:
def createTimeseries(df):
    global ts_t
    ts_P = TimeSeries.from_dataframe(df[['price']])
    ts_covF = TimeSeries.from_dataframe(df[['sen', 'volume', 'trend']])
    
    SPLIT = .8
    # train/test split and scaling of target variable
    ts_train, ts_test = ts_P.split_after(SPLIT)
    
    scalerP = Scaler()
    scalerP.fit_transform(ts_train)
    ts_ttrain = scalerP.transform(ts_train)
    ts_ttest = scalerP.transform(ts_test)
    ts_t = scalerP.transform(ts_P)
    
    # make sure data are of type float
    ts_t = ts_t.astype(np.float32)
    ts_ttrain = ts_ttrain.astype(np.float32)
    ts_ttest = ts_ttest.astype(np.float32)
    
    # train/test split and scaling of feature covariates
    covF_train, covF_test = ts_covF.split_after(SPLIT)
    
    scalerF = Scaler()
    scalerF.fit_transform(covF_train)
    covF_ttrain = scalerF.transform(covF_train)
    covF_ttest = scalerF.transform(covF_test)
    covF_t = scalerF.transform(ts_covF)
    
    # make sure data are of type float
    covF_ttrain = covF_ttrain.astype(np.float32)
    covF_ttest = covF_ttest.astype(np.float32)
    covF_t = covF_t.astype(np.float32)
    
    # # feature engineering - create time covariates: hour, weekday, month, year, country-specific holidays
    # covT = datetime_attribute_timeseries(ts_P.time_index, attribute="day", one_hot=False, add_length=len(covF_ttrain))
    # covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="week"))
    # covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="month"))
    # covT = covT.stack(datetime_attribute_timeseries(covT.time_index, attribute="year"))
    # covT = covT.stack(TimeSeries.from_times_and_values(times=covT.time_index, values=np.arange(len(ts_P) + len(covF_ttrain)), columns=["linear_increase"]))
    # covT = covT.add_holidays(country_code="US")
    # covT = covT.astype(np.float32)
    
    # # train/test split
    # covT_train, covT_test = covT.split_after(ts_train.end_time())
    
    # # rescale the covariates: fitting on the training set
    # scalerT = Scaler()
    # scalerT.fit(covT_train)
    # covT_ttrain = scalerT.transform(covT_train)
    # covT_ttest = scalerT.transform(covT_test)
    # covT_t = scalerT.transform(covT)
    
    # covT_ttrain = covT_ttrain.astype(np.float32)
    # covT_ttest  = covT_ttest.astype(np.float32)
    # covT_t      = covT_t.astype(np.float32)
    
    # # combine feature and time covariates along component dimension: axis=1
    # ts_cov = ts_covF.concatenate( covT.slice_intersect(ts_covF), axis=1 )                      # unscaled F+T
    # cov_t = covF_t.concatenate( covT_t.slice_intersect(covF_t), axis=1 )                       # scaled F+T
    # cov_ttrain = covF_ttrain.concatenate( covT_ttrain.slice_intersect(covF_ttrain), axis=1 )   # scaled F+T training set
    # cov_ttest = covF_ttest.concatenate( covT_ttest.slice_intersect(covF_ttest), axis=1 )       # scaled F+T test set
    
    return ts_ttrain, ts_ttest, cov_t

In [ ]:
# define objective function
def objectiveTFT(trial):
    global mape_, mae_
    # select input and output chunk lengths
    in_len = trial.suggest_int("in_len", 1, 31)
    out_len = trial.suggest_int("out_len", 1, 31)

    # Other hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 8, 32)
    lstm_layers = trial.suggest_categorical("lstm_layers", [1, 2, 3])
    num_attention_heads = trial.suggest_int("num_attention_heads", 2, 8)
    dropout = trial.suggest_float("dropout", 0, 0.3)
    hidden_continuous_size = trial.suggest_int("hidden_continuous_size", 6, 10)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    full_attention = trial.suggest_categorical("full_attention", [True, False])
    lr = trial.suggest_float("lr", 1e-3, 5e-3)
    
    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]

    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }

    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    # build the TFT model
    model = TFTModel(
        input_chunk_length = in_len,
        output_chunk_length = out_len,
        hidden_size = hidden_size,
        lstm_layers = lstm_layers,
        num_attention_heads = num_attention_heads,
        dropout = dropout,
        hidden_continuous_size = hidden_continuous_size,
        use_static_covariates = False,
        batch_size = batch_size,
        optimizer_kwargs = {'lr': lr},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        likelihood = None, 
        loss_fn = torch.nn.MSELoss(),
        full_attention = full_attention,
        torch_metrics = MeanAbsolutePercentageError(),
        random_state = 42069,
        force_reset= True,
        pl_trainer_kwargs = pl_trainer_kwargs,
        add_relative_index= False,
        # add_encoders = None,
        # likelihood=QuantileRegression(quantiles=QUANTILES)
    )

    # train the model
    model.fit(ts_ttrain,             # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)
    
    # Evaluate how good it is on the validation set
    pred = model.predict(len(ts_ttest))
    
    # Calculate Metrics
    mape_ = mape(ts_ttest, pred)
    mae_ = mae(ts_ttest, pred)
    rmse_ = rmse(ts_ttest, pred)

    return rmse_ if rmse_ != np.nan else float("inf")

In [ ]:
timeTFT = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_30WOCovariates.csv")

In [ ]:
timeTFT = []
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])
resultFinale = pd.DataFrame()

results_list = []
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
cov_t = cov_t.astype(np.float32)
# cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

start = time.time()
# optimize hyperparameters by minimizing the sMAPE on the validation set
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objectiveTFT, n_trials=100, callbacks=[print_callback])
timeTFT.append(time.time() - start)

resultFinale = pd.concat([resultFinale, dfResult])
resultFinale.to_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_30WOAnyCovariates.csv")

# Using Best Model

In [ ]:
dfResult = pd.DataFrame()
dfName = ["30", "MAnP", "BP", "BPnP", "MAexperiment", "MAonlyTrend"]

for i, x in enumerate(dfName):
    temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_{x}.csv")
    temp['data'] = x
    temp.set_index("data", inplace = True)
    temp.sort_values(by = 'RMSE', inplace = True)
    dfResult = pd.concat([dfResult, temp.head(5)])

# dfResult.drop(["Unnamed: 0", 'type'], axis = 1, inplace = True)
dfResult.sort_values(by = 'RMSE', inplace = True)
len(dfResult)

In [ ]:
dfResult

In [ ]:
dfResult = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff2/resultTFT_MAonlyTrend.csv")
dfResult = dfResult[dfResult['full_attention'] == False].copy()
dfResult.sort_values(by = 'RMSE', inplace = True)
dfResult.head()

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['trend', 'day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataExperiment.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date', 'trend', 'price', 'volume', 'sen'])

df2 = replaceOutlierMA(df, 30)

ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)
# cov_t = cov_t.astype(np.float32)
cov_t = cov_t[['sen', 'trend', 'day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

In [ ]:
# throughout training we'll monitor the validation loss for both pruning and early stopping
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=10,
    min_delta=0.01,
    mode='min',)

callbacks = [my_stopper]

pl_trainer_kwargs = {
    "accelerator": "auto",
    "callbacks": callbacks,
}

# reproducibility
np.random.seed(42069)
torch.manual_seed(42069)

first_row = dfResult.iloc[0]

# build the TFT model
model = TFTModel(
    input_chunk_length=first_row['in_len'],
    output_chunk_length=first_row['out_len'],
    hidden_size=int(first_row['hidden_size']),
    lstm_layers=first_row['lstm_layers'],
    num_attention_heads=first_row['num_attention_heads'],
    dropout=first_row['dropout'],
    hidden_continuous_size=first_row['hidden_continuous_size'],
    use_static_covariates=True,
    batch_size=int(first_row['batch_size']),
    optimizer_kwargs={'lr': first_row['lr']},
    n_epochs=500,
    nr_epochs_val_period=10,
    likelihood=None,
    loss_fn=torch.nn.MSELoss(),
    full_attention=first_row['full_attention'],
    torch_metrics=MeanAbsolutePercentageError(),
    random_state=42069,
    force_reset=True,
    pl_trainer_kwargs=pl_trainer_kwargs,  # Assuming pl_trainer_kwargs is defined elsewhere
    add_relative_index=False,
)

# train the model
model.fit(ts_ttrain,             # Train Price
          val_series=ts_ttest,          # Val Price
          future_covariates=cov_t,      # Val Covariate
          val_future_covariates=cov_t,  # Val Covariate
          verbose=True)

# Evaluate how good it is on the validation set
pred = model.predict(len(ts_ttest))

# Calculate Metrics
mape_ = mape(ts_ttest, pred)
mae_ = mae(ts_ttest, pred)
rmse_ = rmse(ts_ttest, pred)

In [ ]:
print(f"RMSE : {rmse_}")
print(f"sMAPE: {mape_}")
print(f"MAE : {mae_}")

In [ ]:
print(f"RMSE : {rmse_}")
print(f"sMAPE: {mape_}")
print(f"MAE : {mae_}")

In [ ]:
print(f"RMSE : {rmse_}")
print(f"sMAPE: {mape_}")
print(f"MAE : {mae_}")

## Explainability

In [ ]:
from darts.explainability import TFTExplainer
explainer = TFTExplainer(model)
explainability_result = explainer.explain()
explainer.plot_variable_selection(explainability_result)
# plt.savefig('C:/Users/Razel/Pictures/explainability_plot.png')

In [ ]:
# explainTemp1 = explainability_result.get_encoder_importance()
# explainTemp1['type'] = 'encoder'
# explainTemp2 = explainability_result.get_decoder_importance()
# explainTemp2['type'] = 'decoder'
# explainTemp = pd.concat([explainTemp1, explainTemp2])
# explainTemp.set_index('type').to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Table/autoencoderExplained.xlsx")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="time")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="all")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="heatmap")

In [ ]:
attention = explainability_result.get_attention().mean(axis=1)

time_intersection = ts_ttrain.time_index.intersection(attention.time_index)

ts_ttrain[time_intersection].plot()
attention.plot(label="mean_attention", max_nr_components=12)

# Backtesting

In [ ]:
backTest = model.backtest(series = ts_t,
                          future_covariates = cov_t,
                          start = 0.6,
                          forecast_horizon = int(len(ts_t)*0.1),
                          stride = int(len(ts_t)*0.1),
                          verbose = True,
                          retrain = True,
                          metric = [rmse, mae, mape],
                          reduction = None,
                          last_points_only = False,
                          overlap_end = True
                         )

In [ ]:
backTest

In [ ]:
testing = [backTest[0][0], backTest[1][0], backTest[2][0], backTest[3][0]]

In [ ]:
pd.DataFrame(testing).var()

In [ ]:
dfResult = pd.DataFrame()
data = ['MA', 'MAnP', 'BP', 'BPnP']
model = ['TFT', 'LSTM', 'TCN', 'GRU']

for x in data:
  for y in model:
    temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff2/result{y}_{x}.csv")
    temp.sort_values(by = 'RMSE', inplace = True)
    temp['type'] = y+x
    dfResult = pd.concat([dfResult, temp.iloc[1:5]])

dfResult.set_index('type', inplace = True)

In [ ]:
dfResult

In [ ]:
# backTestResult = []
i = 0
for index, row in dfResult.iterrows():
    print(f"\nStarting-{i}")
    if i < len(backTestResult):
        print('it is skipped')
        i+=1
        continue
    torch.cuda.empty_cache()
    
    df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])

    if index[-2:] == "MA":
        print("It's MA")
        df2 = replaceOutlierMA(df, 30)
        
        ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

        ts_ttrain = ts_ttrain.astype(np.float32)
        ts_ttest = ts_ttest.astype(np.float32)
        cov_t = cov_t.astype(np.float32)
    elif index[-2:] == "BP":
        print("It's BP")
        df2 = replaceOutlierBP(df)
        
        ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

        ts_ttrain = ts_ttrain.astype(np.float32)
        ts_ttest = ts_ttest.astype(np.float32)
        cov_t = cov_t.astype(np.float32)
    elif index[-4:] == "MAnP":
        print("It's MAnP")
        df2 = replaceOutlierMA(df, 30)
        
        ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

        ts_ttrain = ts_ttrain.astype(np.float32)
        ts_ttest = ts_ttest.astype(np.float32)
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)
    elif index[-4:] == "BPnP":
        print("It's BPnP")
        df2 = replaceOutlierBP(df)
        
        ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)

        ts_ttrain = ts_ttrain.astype(np.float32)
        ts_ttest = ts_ttest.astype(np.float32)
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]
    
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    
    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    if index[:3] == 'TFT':
        print("TFT")
        # build the TFT model
        model = TFTModel(
            input_chunk_length=int(row['in_len']),
            output_chunk_length=int(row['out_len']),
            hidden_size=int(row['hidden_size']),
            lstm_layers=int(row['lstm_layers']),
            num_attention_heads=int(row['num_attention_heads']),
            dropout=row['dropout'],
            hidden_continuous_size=int(row['hidden_continuous_size']),
            use_static_covariates=True,
            batch_size=int(row['batch_size']),
            optimizer_kwargs={'lr': row['lr']},
            n_epochs=200,
            nr_epochs_val_period=10,
            likelihood=None,
            loss_fn=torch.nn.MSELoss(),
            full_attention=row['full_attention'],
            torch_metrics=MeanAbsolutePercentageError(),
            random_state=42069,
            force_reset=True,
            pl_trainer_kwargs=pl_trainer_kwargs,  # Assuming pl_trainer_kwargs is defined elsewhere
            add_relative_index=False,
        )
        
        # train the model
        model.fit(ts_ttrain,             # Train Price
                  val_series=ts_ttest,          # Val Price
                  future_covariates=cov_t,      # Val Covariate
                  val_future_covariates=cov_t,  # Val Covariate
                  verbose=True)
        
    elif index[:4] == 'LSTM':
        print("LSTM")
        model = RNNModel(
            model = "LSTM",
            hidden_dim = int(row['hidden_dim']),
            n_rnn_layers = int(row['n_rnn_layers']),
            dropout = row['dropout'],
            batch_size = int(row['batch_size']),
            training_length = int(row['training_length']),
            input_chunk_length = int(row["input_chunk_length"]),
            optimizer_kwargs = {"lr": row['lr']},
            n_epochs = 200,
            nr_epochs_val_period = 10,
            torch_metrics = MeanAbsolutePercentageError(),
            log_tensorboard = True,
            random_state = 42069,
            force_reset = True,
            save_checkpoints = False,
            pl_trainer_kwargs = pl_trainer_kwargs
            )

        # when validating during training, we can use a slightly longer validation
        # set which also contains the first input_chunk_length time steps
        # model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])
    
        # train the model
        model.fit(ts_ttrain,                    # Train Price
                  val_series=ts_ttest,          # Val Price
                  future_covariates=cov_t,      # Val Covariate
                  val_future_covariates=cov_t,  # Val Covariate
                  verbose=True)

    elif index[:3] == 'TCN':
        print("TCN")
        model = TCNModel(
            input_chunk_length = int(row['in_len']),
            output_chunk_length = int(row['out_len']),
            kernel_size = int(row['kernel_size']),
            num_filters = int(row['num_filters']),
            weight_norm = row['weight_norm'],
            dilation_base = int(row['dilation_base']),
            num_layers = int(row['num_layers']),
            dropout = row['dropout'],
            batch_size = int(row['batch_size']),
            optimizer_kwargs = {'lr': row['lr']},
            n_epochs = 200,
            nr_epochs_val_period = 10,
            likelihood = None, 
            loss_fn = torch.nn.MSELoss(),
            torch_metrics = MeanAbsolutePercentageError(),
            random_state = 42069,
            force_reset= True,
            pl_trainer_kwargs = pl_trainer_kwargs,
        )

        # train the model
        model.fit(
            series=ts_ttrain,
            val_series=ts_ttest,
            past_covariates=cov_t,
            val_past_covariates=cov_t,
        )

    else:
        print("GRU")
        model = RNNModel(
            model = "GRU",
            hidden_dim = int(row['hidden_dim']),
            n_rnn_layers = int(row['n_rnn_layers']),
            dropout = row['dropout'],
            batch_size = int(row['batch_size']),
            training_length = int(row['training_length']),
            input_chunk_length = int(row["input_chunk_length"]),
            optimizer_kwargs = {"lr": row['lr']},
            n_epochs = 200,
            nr_epochs_val_period = 10,
            torch_metrics = MeanAbsolutePercentageError(),
            log_tensorboard = True,
            random_state = 42069,
            force_reset = True,
            save_checkpoints = False,
            pl_trainer_kwargs = pl_trainer_kwargs
            )

        # when validating during training, we can use a slightly longer validation
        # set which also contains the first input_chunk_length time steps
        # model_val_set = scaler.transform(series[-(VAL_LEN + in_len) :])
    
        # train the model
        model.fit(ts_ttrain,                    # Train Price
                  val_series=ts_ttest,          # Val Price
                  future_covariates=cov_t,      # Val Covariate
                  val_future_covariates=cov_t,  # Val Covariate
                  verbose=True)

    if index[:3] == 'TCN':
        backTest = model.backtest(series = ts_t,
                                  past_covariates = cov_t,
                                  start = 0.6,
                                  forecast_horizon = int(len(ts_t)*0.1),
                                  stride = int(len(ts_t)*0.1),
                                  verbose = True,
                                  retrain = True,
                                  metric = [rmse, mae],
                                  reduction = None,
                                  last_points_only = False,
                                  overlap_end = True
                                  )
    else:
        backTest = model.backtest(series = ts_t,
                                  future_covariates = cov_t,
                                  start = 0.6,
                                  forecast_horizon = int(len(ts_t)*0.1),
                                  stride = int(len(ts_t)*0.1),
                                  verbose = True,
                                  retrain = True,
                                  metric = [rmse, mae],
                                  reduction = None,
                                  last_points_only = False,
                                  overlap_end = True
                                 )

    backTestResult.append(backTest)
    dfBackTest = pd.DataFrame(backTestResult)
    dfBackTest.to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Backtest Holy/backTest Extra.xlsx")
    i+=1

In [ ]:
backTestResult

In [ ]:
model.fit(ts_ttrain,                    # Train Price
          val_series=ts_ttest,          # Val Price
          future_covariates=cov_t,      # Val Covariate
          val_future_covariates=cov_t,  # Val Covariate
          verbose=True)

In [ ]:
backTest = model.backtest(series = ts_t,
                          future_covariates = cov_t,
                          start = 0.6,
                          forecast_horizon = int(len(ts_t)*0.1),
                          stride = int(len(ts_t)*0.1),
                          verbose = True,
                          retrain = True,
                          metric = [rmse, mae, mape],
                          reduction = None,
                          last_points_only = False,
                          overlap_end = True
                         )

In [ ]:
backTest

In [ ]:
backTest = model.backtest(series = ts_t,
                          future_covariates = cov_t,
                          start = 0.8,
                          forecast_horizon = 102,
                          stride = 102,
                          verbose = True,
                          retrain = True,
                          metric = [rmse, mae, mape],
                          reduction = np.mean,
                          last_points_only = False
                         )

print(backTest)

In [ ]:
dfBackTest.index = dfResult.index

In [ ]:
dfBackTest

In [ ]:
dfBackTest = pd.DataFrame(backTestResult, index=dfResult.index[:len(backTestResult)], columns = ['RMSE', 'MAE', 'MAPE'])
dfBackTest.to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Backtest Holy/backTest.xlsx")

In [ ]:
# model.fit(series=ts_ttrain,             # Train Price
#           val_series=ts_ttest,          # Val Price
#           future_covariates=cov_t,      # Val Covariate
#           val_future_covariates=cov_t,  # Val Covariate

backTest = model.backtest(series = ts_t,
                          future_covariates = cov_t,
                          start = 0.8,
                          forecast_horizon = 102,
                          stride = 102,
                          verbose = True,
                          retrain = True,
                          metric = rmse,
                          reduction = None,
                          last_points_only = False
                         )

In [ ]:
np.mean(backTest)

In [ ]:
backTest.pd_dataframe().to_csv("C:/Users/Razel/Documents/backtestTFT.csv")

In [ ]:
print(darts.metrics.mape(backTest, ts_ttest))
print(darts.metrics.smape(backTest, ts_ttest))
print(darts.metrics.rmse(backTest, ts_ttest))

In [ ]:
print(darts.metrics.mape(pred, ts_ttest))
print(darts.metrics.smape(pred, ts_ttest))
print(darts.metrics.rmse(pred, ts_ttest))

In [ ]:
print(darts.metrics.mape(backTest[0], ts_ttest))
print(darts.metrics.smape(backTest[0], ts_ttest))
print(darts.metrics.rmse(backTest[0], ts_ttest))

# Saved Explainability TFT's Model

## 1st Model

In [ ]:
from darts.explainability import TFTExplainer
explainer = TFTExplainer(model)
explainability_result = explainer.explain()
explainer.plot_variable_selection(explainability_result)
# plt.show()
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Autoencoder Importance.png')

In [ ]:
explainTemp1 = explainability_result.get_encoder_importance()
explainTemp1['type'] = 'encoder'
explainTemp2 = explainability_result.get_decoder_importance()
explainTemp2['type'] = 'decoder'
explainTemp = pd.concat([explainTemp1, explainTemp2])
explainTemp.set_index('type').to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 autoencoderExplained.xlsx")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="time")
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Mean Attention.png')

## 2nd Model

In [ ]:
from darts.explainability import TFTExplainer
explainer = TFTExplainer(model)
explainability_result = explainer.explain()
explainer.plot_variable_selection(explainability_result)
# plt.show()
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Autoencoder Importance.png')

In [ ]:
explainTemp1 = explainability_result.get_encoder_importance()
explainTemp1['type'] = 'encoder'
explainTemp2 = explainability_result.get_decoder_importance()
explainTemp2['type'] = 'decoder'
explainTemp = pd.concat([explainTemp1, explainTemp2])
explainTemp.set_index('type').to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 autoencoderExplained.xlsx")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="time")
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Mean Attention.png')

## 3rd Model

In [ ]:
from darts.explainability import TFTExplainer
explainer = TFTExplainer(model)
explainability_result = explainer.explain()
explainer.plot_variable_selection(explainability_result)
# plt.show()
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Autoencoder Importance.png')

In [ ]:
explainTemp1 = explainability_result.get_encoder_importance()
explainTemp1['type'] = 'encoder'
explainTemp2 = explainability_result.get_decoder_importance()
explainTemp2['type'] = 'decoder'
explainTemp = pd.concat([explainTemp1, explainTemp2])
explainTemp.set_index('type').to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 autoencoderExplained.xlsx")

In [ ]:
explainer.plot_attention(explainability_result, plot_type="time")
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 2 Mean Attention.png')

## 4th Model

In [ ]:
exRes = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/evalTS Diff4/resultTFT_MAdelVol.csv")

In [ ]:
exRes.sort_values(by = "RMSE")

In [ ]:
from darts.explainability import TFTExplainer
explainer = TFTExplainer(model)
explainability_result = explainer.explain()
explainer.plot_variable_selection(explainability_result)
plt.show()
# plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 Autoencoder Importance.png')

In [ ]:
explainTemp1 = explainability_result.get_encoder_importance()
explainTemp1['type'] = 'encoder'
explainTemp2 = explainability_result.get_decoder_importance()
explainTemp2['type'] = 'decoder'
explainTemp = pd.concat([explainTemp1, explainTemp2])
# explainTemp.set_index('type').to_excel("C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 1 autoencoderExplained.xlsx")

In [ ]:
explainTemp = explainTemp.transpose().drop(['year_futcov', 'type'], axis = 0)

In [ ]:
explainTemp.columns = ['encoder', 'decoder']

In [ ]:
plt.bar(explainTemp.index, height = explainTemp['encoder'])
plt.tight_layout()

In [ ]:
explainer.plot_attention(explainability_result, plot_type="time")
plt.savefig('C:/Users/Razel/Documents/GitHub/Skripsi/Model Explainability/Model 2 Mean Attention.png')

# Calculating Time
#### Should've think about this before hand but ohh well wkwkwk

In [ ]:
import time

## TFT

In [ ]:
tipe = "TFT"

dfList2 = ['BP', 'MA']
dfList3 = ['', 'nP']

dfResult = pd.DataFrame()

for x in dfList2:
    for y in dfList3:
        temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/result{tipe}_{x}{y}.csv")
        temp.sort_values(by = 'RMSE', inplace = True)
        temp['type'] = x+y
        temp.set_index('type', inplace = True)
        dfResult = pd.concat([dfResult, temp.head(1)])

dfResult

In [ ]:
timeModel = []
for index, row in dfResult.iterrows():
    df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
    
    if index[:2] == "MA":
        df2 = replaceOutlierMA(df, 30)
    else:
        df2 = replaceOutlierBP(df, 'price')
        df2 = replaceOutlierBP(df2, 'volume')

    if adfuller(df2['price'])[1] > 0.05:
        df2['price'] = df2['price'].diff()
    
    if adfuller(df2['volume'])[1] > 0.05:
        df2['volume'] = df2['volume'].diff()
    
    df2 = df2.iloc[1:].copy()
    
    ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
    ts_ttrain = ts_ttrain.astype(np.float32)
    ts_ttest = ts_ttest.astype(np.float32)
    
    if index[2:] == "nP":
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)
    else:
        cov_t = cov_t.astype(np.float32)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]
    
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    
    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)
    
    # build the TFT model
    model = TFTModel(
        input_chunk_length=row['in_len'],
        output_chunk_length=row['out_len'],
        hidden_size=int(row['hidden_size']),
        lstm_layers=row['lstm_layers'],
        num_attention_heads=row['num_attention_heads'],
        dropout=row['dropout'],
        hidden_continuous_size=row['hidden_continuous_size'],
        use_static_covariates=True,
        batch_size=int(row['batch_size']),
        optimizer_kwargs={'lr': row['lr']},
        n_epochs=500,
        nr_epochs_val_period=10,
        likelihood=None,
        loss_fn=torch.nn.MSELoss(),
        full_attention=row['full_attention'],
        torch_metrics=MeanAbsolutePercentageError(),
        random_state=42069,
        force_reset=True,
        pl_trainer_kwargs=pl_trainer_kwargs,  # Assuming pl_trainer_kwargs is defined elsewhere
        add_relative_index=False,
    )
    
    start = time.time()

    # train the model
    model.fit(ts_ttrain,                    # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)
    
    trainingTime = time.time() - start

    start = time.time()
    pred = model.predict(len(ts_ttest))
    predictingTime = time.time() - start

    timeModel.append([index, trainingTime, predictingTime])

dfTime = pd.DataFrame(timeModel, columns = ['type', 'Training', 'Prediction'])
dfTime.set_index('type', inplace = True)
dfTime['Total Time'] = dfTime.sum(axis = 1)
dfTime.to_excel(f"C:/Users/Razel/Documents/GitHub/Skripsi/time{tipe}.xlsx")

## GRU

In [ ]:
tipe = "GRU"

dfList2 = ['BP', 'MA']
dfList3 = ['', 'nP']

dfResult = pd.DataFrame()

for x in dfList2:
    for y in dfList3:
        temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/result{tipe}_{x}{y}.csv")
        temp.sort_values(by = 'RMSE', inplace = True)
        temp['type'] = x+y
        temp.set_index('type', inplace = True)
        dfResult = pd.concat([dfResult, temp.head(1)])

dfResult

In [ ]:
timeModel = []
for index, row in dfResult.iterrows():
    df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
    
    if index[:2] == "MA":
        df2 = replaceOutlierMA(df, 30)
    else:
        df2 = replaceOutlierBP(df, 'price')
        df2 = replaceOutlierBP(df2, 'volume')

    if adfuller(df2['price'])[1] > 0.05:
        df2['price'] = df2['price'].diff()
    
    if adfuller(df2['volume'])[1] > 0.05:
        df2['volume'] = df2['volume'].diff()
    
    df2 = df2.iloc[1:].copy()
    
    ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
    ts_ttrain = ts_ttrain.astype(np.float32)
    ts_ttest = ts_ttest.astype(np.float32)
    
    if index[2:] == "nP":
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)
    else:
        cov_t = cov_t.astype(np.float32)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]
    
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    
    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)
    
    # build the GRU model
    model = RNNModel(
        model = "GRU",
        hidden_dim = int(row["hidden_dim"]),
        n_rnn_layers = int(row['n_rnn_layers']),
        dropout = row['dropout'],
        batch_size = int(row['batch_size']),
        training_length = int(row['training_length']),
        input_chunk_length = int(row['input_chunk_length']),
        optimizer_kwargs = {"lr": row['lr']},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        torch_metrics = MeanAbsolutePercentageError(),
        log_tensorboard = True,
        random_state = 42069,
        force_reset = True,
        save_checkpoints = False,
        pl_trainer_kwargs = pl_trainer_kwargs
    )

    start = time.time()

    # train the model
    model.fit(ts_ttrain,                    # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)
    
    trainingTime = time.time() - start

    start = time.time()
    pred = model.predict(len(ts_ttest))
    predictingTime = time.time() - start

    timeModel.append([index, trainingTime, predictingTime])

dfTime = pd.DataFrame(timeModel, columns = ['type', 'Training', 'Prediction'])
dfTime.set_index('type', inplace = True)
dfTime['Total Time'] = dfTime.sum(axis = 1)
dfTime.to_excel(f"C:/Users/Razel/Documents/GitHub/Skripsi/time{tipe}.xlsx")

## LSTM

In [ ]:
type = "LSTM"

dfList2 = ['BP', 'MA']
dfList3 = ['', 'nP']

dfResult = pd.DataFrame()

for x in dfList2:
    for y in dfList3:
        temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/result{type}_{x}{y}.csv")
        temp.sort_values(by = 'RMSE', inplace = True)
        temp['type'] = x+y
        temp.set_index('type', inplace = True)
        dfResult = pd.concat([dfResult, temp.head(1)])

dfResult

In [ ]:
timeModel = []
for index, row in dfResult.iterrows():
    df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
    
    if index[:2] == "MA":
        df2 = replaceOutlierMA(df, 30)
    else:
        df2 = replaceOutlierBP(df, 'price')
        df2 = replaceOutlierBP(df2, 'volume')

    if adfuller(df2['price'])[1] > 0.05:
        df2['price'] = df2['price'].diff()
    
    if adfuller(df2['volume'])[1] > 0.05:
        df2['volume'] = df2['volume'].diff()
    
    df2 = df2.iloc[1:].copy()
    
    ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
    
    if index[2:] == "nP":
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)
    else:
        cov_t = cov_t.astype(np.float32)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]
    
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    
    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)
    
    # build the TFT model
    model = RNNModel(
        model = "LSTM",
        hidden_dim = int(row["hidden_dim"]),
        n_rnn_layers = int(row['n_rnn_layers']),
        dropout = row['dropout'],
        batch_size = int(row['batch_size']),
        training_length = int(row['training_length']),
        input_chunk_length = int(row['input_chunk_length']),
        optimizer_kwargs = {"lr": row['lr']},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        torch_metrics = MeanAbsolutePercentageError(),
        log_tensorboard = True,
        random_state = 42069,
        force_reset = True,
        save_checkpoints = False,
        pl_trainer_kwargs = pl_trainer_kwargs
    )
    
    start = time.time()

    # train the model
    model.fit(ts_ttrain,                    # Train Price
              val_series=ts_ttest,          # Val Price
              future_covariates=cov_t,      # Val Covariate
              val_future_covariates=cov_t,  # Val Covariate
              verbose=True)
    
    trainingTime = time.time() - start

    start = time.time()
    pred = model.predict(len(ts_ttest))
    predictingTime = time.time() - start

    timeModel.append([index, trainingTime, predictingTime])

dfTime = pd.DataFrame(timeModel, columns = ['type', 'Training', 'Prediction'])
dfTime.set_index('type', inplace = True)
dfTime['Total Time'] = dfTime.sum(axis = 1)
dfTime.to_excel(f"C:/Users/Razel/Documents/GitHub/Skripsi/time{type}.xlsx")

## TCN

In [ ]:
type = "TCN"

dfList2 = ['BP', 'MA']
dfList3 = ['', 'nP']

dfResult = pd.DataFrame()

for x in dfList2:
    for y in dfList3:
        temp = pd.read_csv(f"C:/Users/Razel/Documents/GitHub/Skripsi/result{type}_{x}{y}.csv")
        temp.sort_values(by = 'RMSE', inplace = True)
        temp['type'] = x+y
        temp.set_index('type', inplace = True)
        dfResult = pd.concat([dfResult, temp.head(1)])

dfResult

In [ ]:
timeModel = []
for index, row in dfResult.iterrows():
    df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
    
    if index[:2] == "MA":
        df2 = replaceOutlierMA(df, 30)
    else:
        df2 = replaceOutlierBP(df, 'price')
        df2 = replaceOutlierBP(df2, 'volume')

    if adfuller(df2['price'])[1] > 0.05:
        df2['price'] = df2['price'].diff()
    
    if adfuller(df2['volume'])[1] > 0.05:
        df2['volume'] = df2['volume'].diff()
    
    df2 = df2.iloc[1:].copy()
    
    ts_ttrain, ts_ttest, cov_t = createTimeseries(df2)
    
    if index[2:] == "nP":
        cov_t = cov_t[['day', 'week', 'month', 'year', 'linear_increase', 'holidays']].astype(np.float32)
    else:
        cov_t = cov_t.astype(np.float32)

    # throughout training we'll monitor the validation loss for both pruning and early stopping
    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=10,
        min_delta=0.01,
        mode='min',)
    
    callbacks = [my_stopper]
    
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": callbacks,
    }
    
    # reproducibility
    np.random.seed(42069)
    torch.manual_seed(42069)

    model = TCNModel(
        input_chunk_length = int(row['in_len']),
        output_chunk_length = int(row['out_len']),
        kernel_size = int(row['kernel_size']),
        num_filters = int(row['num_filters']),
        weight_norm = row['weight_norm'],
        dilation_base = int(row['dilation_base']),
        num_layers = int(row['num_layers']),
        dropout = row['dropout'],
        batch_size = int(row['batch_size']),
        optimizer_kwargs = {'lr': row['lr']},
        n_epochs = 500,
        nr_epochs_val_period = 10,
        likelihood = None, 
        loss_fn = torch.nn.MSELoss(),
        torch_metrics = MeanAbsolutePercentageError(),
        random_state = 42069,
        force_reset= True,
        pl_trainer_kwargs = pl_trainer_kwargs,
    )
    
    start = time.time()

    # train the model
    model.fit(
        series=ts_ttrain,
        val_series=ts_ttest,
        past_covariates=cov_t,
        val_past_covariates=cov_t,
    )
    
    trainingTime = time.time() - start

    start = time.time()
    pred = model.predict(len(ts_ttest))
    predictingTime = time.time() - start

    timeModel.append([index, trainingTime, predictingTime])

dfTime = pd.DataFrame(timeModel, columns = ['type', 'Training', 'Prediction'])
dfTime.set_index('type', inplace = True)
dfTime['Total Time'] = dfTime.sum(axis = 1)
dfTime.to_excel(f"C:/Users/Razel/Documents/GitHub/Skripsi/time{type}.xlsx")

# RANDOM STUFF

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
df = replaceOutlierMA(df, 30)
print(adfuller(df['price']))
print(adfuller(df['volume']))
print(adfuller(df['sen']))
print(adfuller(df['trend']))

In [ ]:
df = pd.read_csv("C:/Users/Razel/Documents/GitHub/Skripsi/finalData/dataRaw.csv", parse_dates=['date'], index_col = ['date'], usecols = ['date',  'price', 'trend', 'volume', 'sen'])
df = replaceOutlierBP(df)
print(adfuller(df['price']))
print(adfuller(df['volume']))
print(adfuller(df['sen']))
print(adfuller(df['trend']))